In [ ]:
import aliases

In [ ]:
###############################################################################
''''''
###############################################################################


from collections import abc as _collabc
import functools as _functools
import inspect as _inspect
import hashlib as _hashlib

from everest.ptolemaic.ousia import Ousia as _Ousia, Blank as _Blank
from everest.utilities import caching as _caching, word as _word


KINDS = dict(zip(
    ('Pos', 'PosKw', 'Args', 'Kw', 'Kwargs'),
    _inspect._ParameterKind,
    ))


class ParamProp:
    
    __slots__ = ('param', 'name', 'default', 'parameter',)

    def __init__(self, param, name, default=_inspect.Parameter.empty, /):
        self.param = param
        self.name, self.default = name, default
        kind = param.kind
        self.parameter = _inspect.Parameter(
            name, KINDS[param.kind], default=default, annotation=param.hint,
            )

    def __get__(self, instance, /, owner=None):
        return getattr(instance.params, self.name)

    def _repr(self):
        return f"name={self.name}, default={self.default}"

    def __repr__(self):
        return f"{type(self).__name__}({self._repr()})"

class Param(metaclass=_Ousia):

    KINDS = KINDS

    kind = None

    __slots__ = ('hint',)

    def __init__(self, hint=_Blank, /):
        if not (kind := self.kind) in self.KINDS:
            raise ValueError(kind)
        if not (isinstance(hint, _Ousia) or isinstance(type(hint), _Ousia)):
            raise TypeError(type(hint))
        self.hint = hint

    def __call__(self, *args, **kwargs):
        return ParamProp(self, *args, **kwargs)

    def __getitem__(self, arg, /):
        return type(self)(self.hint[arg])

    def __repr__(self, /):
        return f"Param.{type(self).__name__}[{repr(self.hint)}]"


class Pos(Param):
    kind = 'Pos'


class PosKw(Param):
    kind = 'PosKw'


class Args(Param):
    kind = 'Args'


class Kw(Param):
    kind = 'Kw'


class Kwargs(Param):
    kind = 'Kwargs'


Param.Pos = Pos()
Param.PosKw = PosKw()
Param.Args = Args()
Param.Kw = Kw()
Param.Kwargs = Kwargs()
Param.Def = Pos()


class Params(_collabc.Mapping):

    __slots__ = ('bound', 'arguments', '_getter', '_softcache')

    def parameterise(self, /, *args, **kwargs):
        return args, kwargs

    signed = False

    def __init__(self, /, *args, **kwargs):
        args, kwargs = self.parameterise(*args, **kwargs)
        bound = self.bound = self.sig.bind(*args, **kwargs)
        bound.apply_defaults()
        self.arguments = bound.arguments

    def __getattr__(self, name):
        if name in (arguments := self.arguments):
            return arguments[name]
        return super().__getattr__(name)

    @_caching.soft_cache()
    def __str__(self):
        args = self.arguments
        return ', '.join(map('='.join, zip(args, map(repr, args.values()))))

    @_caching.soft_cache()
    def __repr__(self):
        return f"{type(self).__name__}({self.__str__()})"

    @property
    @_caching.soft_cache()
    def hashcode(self):
        content = str(self).encode()
        return _hashlib.md5(content).hexdigest()

    @property
    @_caching.soft_cache()
    def hashint(self):
        return int(self.hashcode, 16)

    @property
    @_caching.soft_cache()
    def hashID(self):
        return _word.get_random_english(seed=self.hashint, n=2)

    def __getitem__(self, arg, /):
        return self.arguments[arg]

    def __iter__(self, /):
        return iter(self.arguments)

    def __len__(self, /):
        return len(self.arguments)

    @classmethod
    def __init_subclass__(cls, /, *, sig, parameterise, **kwargs):
        if cls.signed:
            raise TypeError(f"{type(cls)} already signed.")
        cls.sig = sig
        cls.parameterise = parameterise
        cls.signed = True
        super().__init_subclass__(**kwargs)

    @classmethod
    def __class_getitem__(cls, arg, /):
        return type(cls)(
            f"{cls.__name__}[{repr(arg)}]",
            (cls,),
            dict(),
            sig=arg.__signature__,
            parameterise=arg.parameterise,
            )


###############################################################################
###############################################################################


In [ ]:
myparam = Param.Pos

In [ ]:
from everest.ptolemaic.sprites import Intt

In [ ]:
myparam = Param.Pos[Intt]

In [ ]:
intrange = myparam.hint[:10]

In [ ]:
intrange.__repr__

In [ ]:
isinstance(type(intrange), _Ousia)

In [ ]:
myparam[:10]